In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from matplotlib import pyplot as plt
import copy

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")


In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)


testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=10, shuffle=True)
# testloader = torch.utils.data.DataLoader(testset, batch_size=10, shuffle=False)


classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# foreground_classes = {'plane', 'car', 'bird'}

# background_classes = {'cat', 'deer', 'dog', 'frog', 'horse','ship', 'truck'}

# fg1,fg2,fg3 = 0,1,2

In [5]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256,shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=256,shuffle=False)

In [6]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=0)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=0)
    self.conv3 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=0)
    self.fc1 = nn.Linear(512, 256)
    self.fc2 = nn.Linear(256, 64)
    self.fc3 = nn.Linear(64, 10)
    # self.fc4 = nn.Linear(40,10)
    # self.fc5 = nn.Linear(10,2)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    # print(x.shape)
    x = (F.relu(self.conv3(x)))
    x =  x.view(x.size(0), -1)
    # print(x.shape)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = (self.fc3(x))
    # x = F.relu(self.fc4(x))
    # x = self.fc4(x)
    return x

In [7]:
cnn_net = CNN()#.double()
cnn_net = cnn_net.to("cuda")

In [8]:
import torch.optim as optim
criterion_cnn = nn.CrossEntropyLoss()
optimizer_cnn = optim.SGD(cnn_net.parameters(), lr=0.01, momentum=0.9)

In [9]:
acti = []
loss_curi = []
epochs = 300
for epoch in range(epochs): # loop over the dataset multiple times
    ep_lossi = []

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to("cuda"),labels.to("cuda")

        # zero the parameter gradients
        optimizer_cnn.zero_grad()

        # forward + backward + optimize
        outputs = cnn_net(inputs)
        loss = criterion_cnn(outputs, labels)
        loss.backward()
        optimizer_cnn.step()

        # print statistics
        running_loss += loss.item()
        mini_batch = 50
        if i % mini_batch == mini_batch-1:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / mini_batch))
            ep_lossi.append(running_loss/mini_batch) # loss per minibatch
            running_loss = 0.0
            
    if(np.mean(ep_lossi) <= 0.01):
      break;
    loss_curi.append(np.mean(ep_lossi))   #loss per epoch            

print('Finished Training')

[1,    50] loss: 2.306
[1,   100] loss: 2.303
[1,   150] loss: 2.302
[2,    50] loss: 2.300
[2,   100] loss: 2.299
[2,   150] loss: 2.295
[3,    50] loss: 2.212
[3,   100] loss: 2.044
[3,   150] loss: 1.933
[4,    50] loss: 1.746
[4,   100] loss: 1.670
[4,   150] loss: 1.642
[5,    50] loss: 1.560
[5,   100] loss: 1.550
[5,   150] loss: 1.532
[6,    50] loss: 1.451
[6,   100] loss: 1.442
[6,   150] loss: 1.427
[7,    50] loss: 1.361
[7,   100] loss: 1.356
[7,   150] loss: 1.352
[8,    50] loss: 1.308
[8,   100] loss: 1.276
[8,   150] loss: 1.257
[9,    50] loss: 1.217
[9,   100] loss: 1.182
[9,   150] loss: 1.175
[10,    50] loss: 1.154
[10,   100] loss: 1.115
[10,   150] loss: 1.119
[11,    50] loss: 1.094
[11,   100] loss: 1.071
[11,   150] loss: 1.039
[12,    50] loss: 1.022
[12,   100] loss: 1.005
[12,   150] loss: 1.012
[13,    50] loss: 0.993
[13,   100] loss: 0.942
[13,   150] loss: 0.971
[14,    50] loss: 0.932
[14,   100] loss: 0.922
[14,   150] loss: 0.926
[15,    50] loss: 0

In [14]:
torch.save(cnn_net.state_dict(),"/content/drive/My Drive/Research/Cheating_data/CIFAR10_class_classify/weights/"+"cnn_3layer_16_32_32"+".pt")

In [11]:
correct = 0
total = 0
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        images, labels = images.to("cuda"), labels.to("cuda")
        outputs = cnn_net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the %d train images: %d %%' % (total,  100 * correct / total))
print(total,correct)

Accuracy of the network on the 50000 train images: 99 %
50000 49976


In [12]:
correct = 0
total = 0
out = []
pred = []
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to("cuda"),labels.to("cuda")
        out.append(labels.cpu().numpy())
        outputs= cnn_net(images)
        _, predicted = torch.max(outputs.data, 1)
        pred.append(predicted.cpu().numpy())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % ( 100 * correct / total))
print(total,correct)

Accuracy of the network on the 10000 test images: 69 %
10000 6969


In [13]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to("cuda"),labels.to("cuda")
        outputs = cnn_net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 68 %
Accuracy of   car : 76 %
Accuracy of  bird : 76 %
Accuracy of   cat : 40 %
Accuracy of  deer : 61 %
Accuracy of   dog : 40 %
Accuracy of  frog : 66 %
Accuracy of horse : 66 %
Accuracy of  ship : 66 %
Accuracy of truck : 88 %
